Proiect IC : Symmetric-key Corruption Detection : When XOR-MACs Meet Combinatorial Group Testing 

Lidia Istrate 342C4

XOR-GTM with t x m test matrix

In [19]:
def matrix_to_vector_power(H, R):
    t = len(H)
    m = len(H[0])
    v = len(R)
    
    result = []
    for i in range (0, v):
        result.append([])
        for j in range(0, m):
            if len(R[i]) == 1:
                result[i].append(H[R[i][0] - 1][j])
            else:
                xor_result = H[R[i][0] - 1][j] ^ H[R[i][1] - 1][j]
                if len(R[i]) > 2:
                    for k in range(2, len(R[i])):
                        xor_result = xor_result ^ H[R[i][k] - 1][j]
                        
                result[i].append(xor_result)
            
    return result
 




H = [[1, 1, 0, 0], 
     [0, 1, 1, 0],
     [0, 0, 1, 1]]


R = [[1], [2], [3], [1, 2], [2, 3], [1, 2, 3]]

result = matrix_to_vector_power(H, R)
#print(result)

#the bit-wise complement of m, a secure PRF
def Fk(m):
    new_m = []
    for i in range(0, len(m)):
        new_m.append(m[i] ^ 1)
    
    return new_m


#tweakable PRP
def Gkp(m, tweak):
    
    l = int(len(m) / 2)
    key = []
    contor = 0
    
    m_g = []
    
    for i in range(0, len(m)):
       
        if i < l:
            key.append(i ^ tweak)
            contor = i
        elif i == l and len(m) % 2 == 1:
            key.append(i ^ tweak)
        elif i == l and len(m) % 2 == 0:
            key.append(contor ^ tweak)
            contor = contor - 1            
        elif i > l:
            key.append(contor ^ tweak)
            contor = contor - 1

    for i in range(0, len(m)):
        m_g.append(m[i] ^ key[i])
    
    return m_g



def Gkp_inverse(m, tweak):
    

    l = int(len(m) / 2)
    key = []
    contor = 0
    
    m_g = []
    
    for i in range(0, len(m)):
       
        if i < l:
            key.append(i ^ tweak)
            contor = i
        elif i == l and len(m) % 2 == 1:
            key.append(i ^ tweak)
        elif i == l and len(m) % 2 == 0:
            key.append(contor ^ tweak)
            contor = contor - 1            
        elif i > l:
            key.append(contor ^ tweak)
            contor = contor - 1

    for i in range(0, len(m)):
        m_g.append(m[i] ^ key[i])    
    
    return m_g



def xor_gtm_hash(M, H, n):
    S = []
    t = len(H)
    m = len(H[0])
    
    for i in range(0, t):
        S.append([])
        for j in range (0, n):
            S[i].append(0)
        
    for j in range(0, m):
        Z = Fk(M[j])
        for i in range(0, t):
            if H[i][j] == 1:
                for k in range(0, n):
                    S[i][k] = S[i][k] ^ Z[k]
    
    return S



def xor_gtm_tag(M, H, n):
    t = len(H)
    m = len(H[0])
    T = []
    S = xor_gtm_hash(M, H, n)
    
    for i in range(0, t):
        T.append(Gkp(S[i], i))

    return T



def xor_gtm_verify(Mp, Tp, H, n):
    
    Tc = xor_gtm_tag(Mp, H, n)
    tc = len(Tc)
    tp = len(Tp)
    t = 0
    
    if tc == tp:
        t = tc
    else:
        return False
    
    for i in range(0, t):
        for j in range(0, n):
            if Tc[i][j] != Tp[i][j]:
                return False
            
    return True


#print(xor_gtm_verify(Mp, T, H, n))

def xor_gtm_verify_S(Mp, Tp, H, n, R):
    t = len(H)
    m = len(H[0])
    v = len(R)
    
    Sp = []
    for i in range(0, t):
        Sp.append(Gkp_inverse(Tp[i], i))
    
    Sc = xor_gtm_hash(Mp, H, n)
    SpR = matrix_to_vector_power(Sp, R)
    ScR = matrix_to_vector_power(Sc, R)
    
    B = []
    
    for i in range(0, v):
        checked = False
        for j in range(0, n):
            if SpR[i][j] != ScR[i][j]:
                B.append(False)
                checked = True
                break
        if checked == False:
            B.append(True)


    return B

#print(xor_gtm_verify_S(Mp, T, H, n, R))

def xor_gtm_detect(Mp, Tp, H, n, R):
    P = []
    t = len(H)
    m = len(H[0])
    v = len(R)    
    
    for i in range(0, m):
        P.append(i)
        
    Sp = []
    for i in range(0, t):
        Sp.append(Gkp_inverse(Tp[i], i))   
        
    Sc = xor_gtm_hash(Mp, H, n)
    SpR = matrix_to_vector_power(Sp, R)
    ScR = matrix_to_vector_power(Sc, R)   
    HR = matrix_to_vector_power(H, R)

    #print("Sp ", Sp, " Sc ", Sc)
    #print("SpR ", SpR, " ScR ", ScR)
    
    for i in range(0, v):
        checked = False
        for j in range(0, n):
            if SpR[i][j] != ScR[i][j]:
                checked = True
                break
        if checked == False and i in P:
            P.remove(i)   
    
    
    return P



M = [[1, 0, 1, 1], [1, 1, 1, 1], [0, 0, 0, 1], [0, 1, 1, 0]]
Mp = [[1, 0, 1, 1], [1, 1, 1, 1], [0, 0, 0, 1], [0, 1, 1, 0]]

#Mp = [[0, 0, 1, 1], [1, 0, 1, 1], [0, 0, 0, 1], [0, 1, 1, 0]]

t = len(H)
m = len(H[0])
n = len(M[0])
v = len(R)
T = xor_gtm_tag(M, H, n)
print(xor_gtm_verify(Mp, T, H, n))
print(xor_gtm_detect(Mp, T, H, n, R))

True
[]


XOR-GTM-PPI

In [20]:
from random import randrange

def xor_gtm_ppi_hash(M, HR, n, w, t, m):
    H = []
    for i in range(0, t):
        H.append(HR[i])
        
    for i in range(0, w):
        if i >= len(HR):
            HR.append([])
            for j in range(0, len(HR[0])):
                HR[i].append(0)
        HR[i][1] += 1
    
    S = []
    for i in range(0, t):
        S.append([])
        for j in range (0, n):
            S[i].append(0)   

            

    
    for j in range(0, m):
        Z = Fk(M[j])
        I = []
        for k in range(0, w):
            I.append((HR[k][1] + (j - 1)) % t) # %m
            
        for i in I:
            for l in range(0, n):
                S[i][l] = S[i][l] ^ Z[l]
                    
    return S
    


def xor_gtm_ppi_tag(M, H, n, t, m, w):
    T = []
    HR = matrix_to_vector_power(H, R)
    S = xor_gtm_ppi_hash(M, HR, n, w, t, m)
    
    for i in range(0, t):
        T.append(Gkp(S[i], i))

    return T



def xor_gtm_ppi_detect(Mp, Tp, H, w, m, t, v, HR, R):
    for i in range(0, w):
        H[i][1] += 1
        #H[1][i] = randrange(m)    
        
    P = []
    
    for i in range(0, m):
        P.append(i)
        
    Sp = []
    for i in range(0, t):
        Sp.append(Gkp_inverse(Tp[i], i))   
        
    Sc = xor_gtm_ppi_hash(M, HR, n, w, t, m)
    SpR = matrix_to_vector_power(Sp, R)
    ScR = matrix_to_vector_power(Sc, R)   

    
    for i in range(0, v):
        checked = False
        for j in range(0, n):
            if SpR[i][j] != ScR[i][j]:
                checked = True
                break
        if checked == False:
            S = []
            for j in range(0, w):
                S.append((H[1][j] + (i - 1)) % m) 
                
            for e in S:
                if e in P:
                    P.remove(e)   
    
    return P   

HR = matrix_to_vector_power(H, R)

w = int(pow(2, 4)) + 1
#w = 2
T = xor_gtm_ppi_tag(M, H, n, t, m, w)
w = 2
print(xor_gtm_ppi_detect(Mp, T, H, w, m, t, v, HR, R))



[0, 2]
